In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading the data

In [ ]:
train = pd.read_csv('data/ML/ML_train.csv')
test = pd.read_csv('data/ML/ML_test.csv')

In [ ]:
train.head()

,Text,Sentiment
0,watched 34 movie wondered got horrible review ...,0
1,have never understood appeal garbo always come...,0
2,nacho vigalondo is famous spain is kind bad sh...,0
3,eric idle robbie coltraine janet suzman should...,0
4,great storyline message joan plowright is supe...,1


In [ ]:
trial_train = train.copy()
trial_train.head()

,Text,Sentiment
0,watched 34 movie wondered got horrible review ...,0
1,have never understood appeal garbo always come...,0
2,nacho vigalondo is famous spain is kind bad sh...,0
3,eric idle robbie coltraine janet suzman should...,0
4,great storyline message joan plowright is supe...,1


In [ ]:
class2_X_train = trial_train['Text']
class2_y_train = trial_train['Sentiment']

In [ ]:
trial_test = test.copy()
trial_test.head()

,Text,Sentiment
0,others have already commented decline director...,0
1,play is cleverly constructed begin porter rain...,1
2,gettin sick movie sound entertaining oneline s...,0
3,ruin will brief is great acting funny line att...,1
4,knew would gave rent laugh maybe mindless fun ...,0


In [ ]:
class2_X_test = trial_test['Text']
class2_y_test = trial_test['Sentiment']

# Vectorization

BOW Vectorizer

In [ ]:
bow_vectorizer = CountVectorizer()
bow_vectorizer.fit(class2_X_train)

bow_X_train = bow_vectorizer.transform(class2_X_train)
bow_X_test = bow_vectorizer.transform(class2_X_test)

TF-IDF Vectorizer

In [ ]:
# ngram_range=(1, 3): This tells the vectorizer to consider unigrams, bigrams, and trigrams
# min_df=2: This means an n-gram must appear in at least two documents to be considered. This helps in removing very rare n-grams that might not be useful for modeling.
# max_df=0.85: This means an n-gram appearing in more than 85% of the documents will be ignored, helping in filtering out too common n-grams.

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=2, max_df=0.85)
tfidf_vectorizer.fit(X_train)

tfidf_X_train = tfidf_vectorizer.transform(X_train)
tfidf_X_test = tfidf_vectorizer.transform(X_test)

# Function to help us evaluate model performance

In [ ]:
def train_and_eval(model, trainX, trainY, testX, testY):

    # training the model
    fitted_model = model.fit(trainX, trainY)

    # getting predictions
    y_preds_train = fitted_model.predict(trainX)
    y_preds_test = fitted_model.predict(testX)

    # evaluating the model
    print()
    print(model)
    print(f"Train accuracy score : {accuracy_score(trainY, y_preds_train)}")
    print(f"Test accuracy score : {accuracy_score(testY, y_preds_test)}")
    print(classification_report(testY, y_preds_test))
    print('\n',40*'-')

## Logistic Regression Hyperparameter Tuning


Logistic Regression with BOW


In [ ]:
#Selections of param grid are based on previous mini hyperparameter tuning to find better C values 
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],
    'max_iter': [400]
}

In [ ]:
log_model = LogisticRegression(random_state = 42, n_jobs = -1)
clf = GridSearchCV(log_model, param_grid=param_grid, cv=3, verbose=4, scoring='accuracy')

In [ ]:
best_clf = clf.fit(bow_X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END C=0.1, max_iter=400, penalty=l1, solver=liblinear;, score=0.879 total time=   2.2s
[CV 2/3] END C=0.1, max_iter=400, penalty=l1, solver=liblinear;, score=0.875 total time=   1.5s
[CV 3/3] END C=0.1, max_iter=400, penalty=l1, solver=liblinear;, score=0.878 total time=   1.3s
[CV 1/3] END C=0.1, max_iter=400, penalty=l2, solver=liblinear;, score=0.888 total time=  15.6s
[CV 2/3] END C=0.1, max_iter=400, penalty=l2, solver=liblinear;, score=0.885 total time=   9.5s
[CV 3/3] END C=0.1, max_iter=400, penalty=l2, solver=liblinear;, score=0.890 total time=   7.0s
[CV 1/3] END C=1, max_iter=400, penalty=l1, solver=liblinear;, score=0.874 total time=   1.6s
[CV 2/3] END C=1, max_iter=400, penalty=l1, solver=liblinear;, score=0.876 total time=   2.1s
[CV 3/3] END C=1, max_iter=400, penalty=l1, solver=liblinear;, score=0.876 total time=   1.9s
[CV 1/3] END C=1, max_iter=400, penalty=l2, solver=liblinear;, score=0.883 total t

In [ ]:
best_clf.best_params_

{'C': 0.1, 'max_iter': 400, 'penalty': 'l2', 'solver': 'liblinear'}

In [ ]:
best_clf.best_score_

0.8873499923593785

In [ ]:
best_log = LogisticRegression(**best_clf.best_params_)

In [ ]:
train_and_eval(best_log, bow_X_train, y_train, bow_X_test, y_test)


LogisticRegression(C=0.1, max_iter=400, solver='liblinear')
Train accuracy score : 0.96795
Test accuracy score : 0.8924892489248925
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      5000
           1       0.89      0.90      0.89      4999

    accuracy                           0.89      9999
   macro avg       0.89      0.89      0.89      9999
weighted avg       0.89      0.89      0.89      9999


 ----------------------------------------


Best Logistic Regression Model with BOW gives us 89.2% accuracy on test data

Logistic Regression with TF-IDF


In [ ]:
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [400]
}

In [ ]:
log_model = LogisticRegression(random_state = 42, n_jobs = -1)
clf = RandomizedSearchCV(log_model, param_distributions=param_grid, cv=3, verbose=4, scoring='accuracy', random_state=42, n_iter=20)


In [ ]:
best_clf = clf.fit(bow_X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END C=0.1, max_iter=400, penalty=l1, solver=liblinear;, score=0.879 total time=   1.3s
[CV 2/3] END C=0.1, max_iter=400, penalty=l1, solver=liblinear;, score=0.875 total time=   1.3s
[CV 3/3] END C=0.1, max_iter=400, penalty=l1, solver=liblinear;, score=0.878 total time=   1.4s
[CV 1/3] END C=0.1, max_iter=400, penalty=l1, solver=saga;, score=0.878 total time= 1.9min
[CV 2/3] END C=0.1, max_iter=400, penalty=l1, solver=saga;, score=0.873 total time= 2.0min
[CV 3/3] END C=0.1, max_iter=400, penalty=l1, solver=saga;, score=0.878 total time= 1.9min
[CV 1/3] END C=0.1, max_iter=400, penalty=l2, solver=liblinear;, score=0.888 total time=   8.6s
[CV 2/3] END C=0.1, max_iter=400, penalty=l2, solver=liblinear;, score=0.884 total time=  10.4s
[CV 3/3] END C=0.1, max_iter=400, penalty=l2, solver=liblinear;, score=0.890 total time=  10.4s
[CV 1/3] END C=0.1, max_iter=400, penalty=l2, solver=saga;, score=0.888 total time=  49.6s

In [ ]:
log_model = LogisticRegression(random_state = 42, n_jobs = -1)
clf = RandomizedSearchCV(log_model, param_distributions=param_grid, cv=3, verbose=4, scoring='accuracy', random_state=42, n_iter=20)

In [ ]:
%%time
best_clf = clf.fit(tfidf_X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END C=0.1, max_iter=400, penalty=l1, solver=liblinear;, score=0.766 total time=  18.2s
[CV 2/3] END C=0.1, max_iter=400, penalty=l1, solver=liblinear;, score=0.765 total time=  15.9s
[CV 3/3] END C=0.1, max_iter=400, penalty=l1, solver=liblinear;, score=0.772 total time=  13.9s
[CV 1/3] END C=0.1, max_iter=400, penalty=l1, solver=saga;, score=0.765 total time=  20.3s
[CV 2/3] END C=0.1, max_iter=400, penalty=l1, solver=saga;, score=0.765 total time=  17.4s
[CV 3/3] END C=0.1, max_iter=400, penalty=l1, solver=saga;, score=0.772 total time=  16.3s
[CV 1/3] END C=0.1, max_iter=400, penalty=l2, solver=liblinear;, score=0.854 total time=   3.7s
[CV 2/3] END C=0.1, max_iter=400, penalty=l2, solver=liblinear;, score=0.854 total time=   4.7s
[CV 3/3] END C=0.1, max_iter=400, penalty=l2, solver=liblinear;, score=0.859 total time=   4.8s
[CV 1/3] END C=0.1, max_iter=400, penalty=l2, solver=saga;, score=0.854 total time=  15.0s

In [ ]:
best_clf.best_params_

{'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 400, 'C': 10}

In [ ]:
best_clf.best_score_

0.9037249386311904

In [ ]:
best_log = LogisticRegression(**best_clf.best_params_)

In [ ]:
train_and_eval(best_log, tfidf_X_train, y_train, tfidf_X_test, y_test)


LogisticRegression(C=10, max_iter=400, solver='liblinear')
Train accuracy score : 0.999175
Test accuracy score : 0.9062906290629063
              precision    recall  f1-score   support

           0       0.91      0.90      0.91      5000
           1       0.90      0.91      0.91      4999

    accuracy                           0.91      9999
   macro avg       0.91      0.91      0.91      9999
weighted avg       0.91      0.91      0.91      9999


 ----------------------------------------


Best Logistic Regression Model with TFIDF gives us 90.6% accuracy on test data

## Random Forest Hyperparameter Tuning


Random Forest Classifier with BOW

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [20, 30, 50, 75, 100],
    'min_samples_split': [10, 25, 50, 100, 150, 200],
    'min_samples_leaf': [25, 50, 100]
}

In [ ]:
rf_model = RandomForestClassifier(random_state = 42, n_jobs = -1)
clf = RandomizedSearchCV(rf_model, param_distributions=param_grid, cv=3, verbose=4, scoring='accuracy', random_state=42)

In [ ]:
best_clf = clf.fit(bow_X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END criterion=log_loss, max_depth=50, min_samples_leaf=50, min_samples_split=100, n_estimators=100;, score=0.847 total time=  23.6s
[CV 2/3] END criterion=log_loss, max_depth=50, min_samples_leaf=50, min_samples_split=100, n_estimators=100;, score=0.835 total time=  19.5s
[CV 3/3] END criterion=log_loss, max_depth=50, min_samples_leaf=50, min_samples_split=100, n_estimators=100;, score=0.843 total time=  17.0s
[CV 1/3] END criterion=entropy, max_depth=100, min_samples_leaf=50, min_samples_split=150, n_estimators=50;, score=0.834 total time=  11.0s
[CV 2/3] END criterion=entropy, max_depth=100, min_samples_leaf=50, min_samples_split=150, n_estimators=50;, score=0.821 total time=   9.4s
[CV 3/3] END criterion=entropy, max_depth=100, min_samples_leaf=50, min_samples_split=150, n_estimators=50;, score=0.838 total time=   7.9s
[CV 1/3] END criterion=log_loss, max_depth=100, min_samples_leaf=50, min_samples_split=10, n_est

In [ ]:
best_clf.best_params_

{'n_estimators': 250,
 'min_samples_split': 10,
 'min_samples_leaf': 50,
 'max_depth': 100,
 'criterion': 'log_loss'}

In [ ]:
best_clf.best_score_

0.8492000173110673

In [ ]:
best_rf = RandomForestClassifier(**best_clf.best_params_)


In [ ]:
train_and_eval(best_rf, bow_X_train, y_train, bow_X_test, y_test)



RandomForestClassifier(criterion='log_loss', max_depth=100, min_samples_leaf=50,
                       min_samples_split=10, n_estimators=250)
Train accuracy score : 0.86275
Test accuracy score : 0.8525852585258525
              precision    recall  f1-score   support

           0       0.87      0.83      0.85      5000
           1       0.83      0.88      0.86      4999

    accuracy                           0.85      9999
   macro avg       0.85      0.85      0.85      9999
weighted avg       0.85      0.85      0.85      9999


 ----------------------------------------


Best Random Forest Classifier with BOW gives us 85.3% accuracy on test data

Random Forest with TF-IDF

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150, 200, 500],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [20, 30, 50, 75, 100],
    'min_samples_split': [10, 25, 50, 100, 150, 200, 250],
    'min_samples_leaf': [25, 50, 100]
}

In [ ]:
rf_model = RandomForestClassifier(random_state = 42, n_jobs = -1)
clf = RandomizedSearchCV(rf_model, param_distributions=param_grid, cv=3, verbose=4, scoring='accuracy', random_state=42)

In [ ]:
best_clf = clf.fit(tfidf_X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END criterion=log_loss, max_depth=20, min_samples_leaf=100, min_samples_split=25, n_estimators=100;, score=0.811 total time=  28.1s
[CV 2/3] END criterion=log_loss, max_depth=20, min_samples_leaf=100, min_samples_split=25, n_estimators=100;, score=0.820 total time=  25.4s
[CV 3/3] END criterion=log_loss, max_depth=20, min_samples_leaf=100, min_samples_split=25, n_estimators=100;, score=0.817 total time=  20.1s
[CV 1/3] END criterion=log_loss, max_depth=75, min_samples_leaf=100, min_samples_split=150, n_estimators=500;, score=0.842 total time= 1.4min
[CV 2/3] END criterion=log_loss, max_depth=75, min_samples_leaf=100, min_samples_split=150, n_estimators=500;, score=0.844 total time= 1.5min
[CV 3/3] END criterion=log_loss, max_depth=75, min_samples_leaf=100, min_samples_split=150, n_estimators=500;, score=0.842 total time= 1.4min
[CV 1/3] END criterion=entropy, max_depth=75, min_samples_leaf=25, min_samples_split=150, 

In [ ]:
# best parameters
best_clf.best_params_

{'n_estimators': 500,
 'min_samples_split': 250,
 'min_samples_leaf': 25,
 'max_depth': 50,
 'criterion': 'log_loss'}

In [ ]:
# highest accuracy score
best_clf.best_score_

0.8643249773309729

In [ ]:
best_rf = RandomForestClassifier(**best_clf.best_params_)

In [ ]:
train_and_eval(best_rf, tfidf_X_train, y_train, tfidf_X_test, y_test)


RandomForestClassifier(criterion='log_loss', max_depth=50, min_samples_leaf=25,
                       min_samples_split=250, n_estimators=500)
Train accuracy score : 0.884975
Test accuracy score : 0.863986398639864
              precision    recall  f1-score   support

           0       0.89      0.84      0.86      5000
           1       0.85      0.89      0.87      4999

    accuracy                           0.86      9999
   macro avg       0.87      0.86      0.86      9999
weighted avg       0.87      0.86      0.86      9999


 ----------------------------------------


Best Random forest Classifier with TF-IDF gives us 86.3% accuracy on test data

# Multinomial Naive Bayes Hyperparameter Tuning

Multinomial Naive Bayes with BOW

In [ ]:
param_grid = {
    'alpha': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'force_alpha': [True, False],
    'fit_prior': [True, False]
}

In [ ]:
nb_model = MultinomialNB()
clf = GridSearchCV(nb_model, 
                   param_grid=param_grid, 
                   cv=5, 
                   verbose=4,
                   scoring='accuracy')

In [ ]:
best_clf = clf.fit(bow_X_train, y_train)

Fitting 5 folds for each of 44 candidates, totalling 220 fits
[CV 1/5] END alpha=0.0, fit_prior=True, force_alpha=True;, score=0.579 total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:907: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:907: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


[CV 2/5] END alpha=0.0, fit_prior=True, force_alpha=True;, score=0.582 total time=   0.1s
[CV 3/5] END alpha=0.0, fit_prior=True, force_alpha=True;, score=0.582 total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:907: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:907: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


[CV 4/5] END alpha=0.0, fit_prior=True, force_alpha=True;, score=0.577 total time=   0.1s
[CV 5/5] END alpha=0.0, fit_prior=True, force_alpha=True;, score=0.582 total time=   0.1s


...

[CV 3/5] END alpha=0.0, fit_prior=False, force_alpha=True;, score=0.582 total time=   0.1s
[CV 4/5] END alpha=0.0, fit_prior=False, force_alpha=True;, score=0.577 total time=   0.1s
[CV 5/5] END alpha=0.0, fit_prior=False, force_alpha=True;, score=0.582 total time=   0.1s
[CV 1/5] END alpha=0.0, fit_prior=False, force_alpha=False;, score=0.788 total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


[CV 2/5] END alpha=0.0, fit_prior=False, force_alpha=False;, score=0.778 total time=   0.1s
[CV 3/5] END alpha=0.0, fit_prior=False, force_alpha=False;, score=0.779 total time=   0.1s
[CV 4/5] END alpha=0.0, fit_prior=False, force_alpha=False;, score=0.783 total time=   0.1s
[CV 5/5] END alpha=0.0, fit_prior=False, force_alpha=False;, score=0.787 total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


[CV 1/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.859 total time=   0.1s
[CV 2/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.853 total time=   0.1s
[CV 3/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.855 total time=   0.1s
[CV 4/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.857 total time=   0.1s
[CV 5/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.859 total time=   0.1s
[CV 1/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.859 total time=   0.1s
[CV 2/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.853 total time=   0.1s
[CV 3/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.855 total time=   0.1s
[CV 4/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.857 total time=   0.1s
[CV 5/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.859 total time=   0.1s
[CV 1/5] END alpha=0.1, fit_prior=False, force_alpha=True;, score=0.859 total time=   0.1s
[CV 

In [ ]:
# best parameters
best_clf.best_params_

{'alpha': 1.0, 'fit_prior': True, 'force_alpha': True}

In [ ]:
# highest accuracy score
best_clf.best_score_

0.859975

In [ ]:
best_nb = MultinomialNB(**best_clf.best_params_)

In [ ]:
train_and_eval(best_nb, bow_X_train, y_train, bow_X_test, y_test)


MultinomialNB(force_alpha=True)
Train accuracy score : 0.916925
Test accuracy score : 0.8603860386038604
              precision    recall  f1-score   support

           0       0.85      0.88      0.86      5000
           1       0.87      0.84      0.86      4999

    accuracy                           0.86      9999
   macro avg       0.86      0.86      0.86      9999
weighted avg       0.86      0.86      0.86      9999


 ----------------------------------------


Best Naive Bayes model with BOW gives us 86.0% accuracy on test data

Multinomial Naive Bayes with TF-IDF

In [ ]:
param_grid = {
    'alpha': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'force_alpha': [True, False],
    'fit_prior': [True, False]
}

In [ ]:
nb_model = MultinomialNB()
clf = GridSearchCV(nb_model, param_grid=param_grid, cv=5, verbose=4, scoring='accuracy')

In [ ]:
best_clf = clf.fit(tfidf_X_train, y_train)

Fitting 5 folds for each of 44 candidates, totalling 220 fits


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:907: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


[CV 1/5] END alpha=0.0, fit_prior=True, force_alpha=True;, score=0.504 total time=   0.2s


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:907: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


[CV 2/5] END alpha=0.0, fit_prior=True, force_alpha=True;, score=0.505 total time=   0.2s


...

[CV 3/5] END alpha=0.0, fit_prior=False, force_alpha=False;, score=0.848 total time=   0.2s


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


[CV 4/5] END alpha=0.0, fit_prior=False, force_alpha=False;, score=0.843 total time=   0.2s


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


[CV 5/5] END alpha=0.0, fit_prior=False, force_alpha=False;, score=0.846 total time=   0.2s
[CV 1/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.902 total time=   0.2s
[CV 2/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.895 total time=   0.2s
[CV 3/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.899 total time=   0.2s
[CV 4/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.896 total time=   0.2s
[CV 5/5] END alpha=0.1, fit_prior=True, force_alpha=True;, score=0.902 total time=   0.2s
[CV 1/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.902 total time=   0.2s
[CV 2/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.895 total time=   0.2s
[CV 3/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.899 total time=   0.2s
[CV 4/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.896 total time=   0.2s
[CV 5/5] END alpha=0.1, fit_prior=True, force_alpha=False;, score=0.902 total time=   0.2s
[CV

In [ ]:
# best parameters
best_clf.best_params_

{'alpha': 0.2, 'fit_prior': True, 'force_alpha': True}

In [ ]:
# highest accuracy score
best_clf.best_score_

0.8991750000000002

In [ ]:
best_nb = MultinomialNB(**best_clf.best_params_)

In [ ]:
train_and_eval(best_nb, tfidf_X_train, y_train, tfidf_X_test, y_test)


MultinomialNB(alpha=0.2, force_alpha=True)
Train accuracy score : 0.988425
Test accuracy score : 0.8928892889288929
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      5000
           1       0.88      0.90      0.89      4999

    accuracy                           0.89      9999
   macro avg       0.89      0.89      0.89      9999
weighted avg       0.89      0.89      0.89      9999


 ----------------------------------------


Best Multinomial Naive Bayes gives us an accuracy of 89.2% on test data